In [1]:
import pandas as pd
import csv
import pymssql

In [2]:
# Connect to MSSQL database
server = 'localhost'
database = 'BockBluster'
username = 'sa'
password = 'thisIsSuperStrong1234'

connection = pymssql.connect(server=server, database=database, user=username, password=password)
cursor = connection.cursor()

# Define the SQL query
query = "SELECT * FROM genre"

# Execute the query
cursor.execute(query)

# Fetch all the rows
rows = cursor.fetchall()

# Create an empty list
data_list = []

# Iterate over the rows and extract values into the list
for row in rows:
    value1 = row[0]
    value2 = row[1]
    dictionary = {'id': value1, 'name': value2}
    data_list.append(dictionary)

# Close the cursor and the connection
cursor.close()
connection.close()

In [3]:
df = pd.read_csv('../data/movies.csv')

In [4]:
df['genre'] = df['genre'].apply(lambda x: x.split(', '))

df['genre'] = df['genre'].apply(lambda x: list(set(x)))

df_new = df[['id', 'genre']]

df_new = df_new.explode('genre')

In [5]:
movies_list = df_new.to_dict(orient='records')


mapping = []
for movie in movies_list:
    movie_id = movie['id']
    genre_name = movie['genre']
    for genre in data_list:
        if genre['name'] == genre_name:
            genre_id = genre['id']
            new_movie = {'movie_id': movie_id, 'genre_id': genre_id}
            mapping.append(new_movie)
            break

In [6]:
# Connect to MSSQL database
server = 'localhost'
database = 'BockBluster'
username = 'sa'
password = 'thisIsSuperStrong1234'

connection = pymssql.connect(server=server, database=database, user=username, password=password)

# Insert dictionaries into the table
cursor = connection.cursor()
for dictionary in mapping:
    cursor.execute("INSERT INTO movie_genre (movie_id, genre_id) VALUES (%s, %s)", (dictionary['movie_id'], dictionary['genre_id']))

# Commit the changes and close the connection
connection.commit()
connection.close()